In [1]:
#hide
#default_exp tools.query
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Query

> Shows results for queried experiments.

In [2]:
#export
import warnings
warnings.filterwarnings('ignore')

import argparse
import sys
sys.path.append('.')
from collections import namedtuple
from IPython.display import display
import pandas as pd

# hpsearch api
import hpsearch.utils.experiment_utils as ut
import hpsearch.config.hp_defaults as dflt

In [3]:
#for tests
import pytest
import os

import hpsearch.config.hp_defaults as dflt
from hpsearch.examples.dummy_experiment_manager import generate_data

## query

In [4]:
# export
def query (pv={}, pf={}, pall=[], pexact=False, folder=None, 
           metric=None, experiments=None, runs=None, op=None, stats=['mean'], 
           results=0, other_parameters=False):
    
    
    result_query = ut.query(folder_experiments=folder, score_name=''+metric, experiments=experiments,
                        run_number=runs, parameters_fixed=pf, parameters_variable=pv, parameters_all = pall, exact_match=pexact, 
                        ascending=op=='min', stats=stats, min_results=results, query_other_parameters=other_parameters)
    
    if not other_parameters:
        result_query = result_query[1]
        result_query = result_query['stats']

    return result_query 

## do_query_and_show

In [5]:
#export
def do_query_and_show (pall=[], best=None, compact=0, exact=False, experiments=None, pf={}, last=None, 
                       metric=None, op=None, other_parameters=False, input_range=None, results=0, 
                       folder=None, round=2, runs=None, show=False, stats=['mean'], pv={},
                       sort=None, display_all_columns=False, col_width=None, 
                       manager_path=dflt.manager_path):
    
    from hpsearch.config.hpconfig import get_experiment_manager
    em = get_experiment_manager (manager_path=manager_path)
    if folder is not None or metric is not None or op is not None:
        if folder is not None: em.set_path_experiments (folder=folder)
        if metric is not None: em.key_score = metric
        if op is not None: em.op = op       
        
    df = query (pv=pv, pf=pf, pall=pall, pexact=exact, folder=em.folder, 
               metric=em.key_score, experiments=experiments, runs=runs, op=em.op, stats=stats, 
               results=results, other_parameters=other_parameters)
    df = ut.replace_with_default_values (df)
    if sort is not None:
        stats_cols = df[dflt.stats_col].columns.get_level_values(1)
        pars_cols = df[dflt.parameters_col].columns.get_level_values(0)
        if sort in stats_cols:
            score_name_sort = df[dflt.stats_col].columns.get_level_values(0).unique()
            if len(score_name_sort)>1: print (f'sorting using first score_name from {score_name_sort}')
            score_name_sort = score_name_sort[0]
            sort_col = (dflt.stats_col, score_name_sort, sort)
        elif sort in pars_col:
            sort_col = (dflt.parameters_col, sort, '')
        else:
            raise ValueError (f'sort must be in {stats_cols.tolist()+pars_cols.tolist()}')
        df = df.sort_values(by=sort_col, ascending=(em.op=='min'))
    if experiments is None:
        experiments = []
    if last is not None:
        experiments += range(df.index.max()-last+1, df.index.max()+1)
    if best is not None:
        experiments += list(df.index[:best])
    if input_range is not None:
        assert len(input_range) == 2
        experiments += range(input_range[0], input_range[1])
    if len(experiments) > 0: 
        df = df.loc[[x for x in df.index if x in experiments]]
    
    if col_width is not None:
        pd.set_option('max_colwidth', col_width)
    
    if (round is not None) and (round != 0):
        stats_col = pd.MultiIndex.from_product ([[dflt.stats_col], 
                                                 df[dflt.stats_col].columns.get_level_values(0).unique(),
                                                 stats])
        df[stats_col] = df[stats_col].round(round)
    if display_all_columns:
        display (df)
    
    print (f'experiments: {list(df.index)}')
    print (f'min experiment #: {df.index.min()}, max experiment #: {df.index.max()}')

    print ('result of query:')
    _, df2 = ut.get_parameters_unique(df)
    #df2.index.name = 'experiment #'
    if compact > 0:
        prev_cols = df2.columns.copy()
        df2, dict_rename = ut.compact_parameters (df2, compact)
        for k, kor in zip(df2.columns, prev_cols):
            print (f'{k} => {kor}')
    display (df2)
            
    if show:
        import hpsearch.visualization.plot_visdom as pv
        pv.plot_multiple_histories(df.index, folder=em.folder,metrics=em.key_score, parameters=None)
    return df2

### Run simple query

Run query without any condition, retrieving all the data

In [6]:
#export tests.tools.test_query
def test_do_query_and_show ():
    path_results = 'do_query_and_show'
    em = generate_data (path_results)
    
    df=do_query_and_show (manager_path=em.manager_path)
    assert sorted(os.listdir (f'test_{path_results}/default/managers'))==['fields', 'info', 'logs.txt', 'whole']
    assert sorted(os.listdir (f'test_{path_results}/default/managers/whole'))==['DummyExperimentManager-default.pk', 'last.pk']
    par = lambda parameter: (dflt.parameters_col, parameter, '')
    assert (df[par('epochs')] == [15,30,5,15,30,15,5,30,5]).all()
    assert (df[par('offset')] == [.6,.6,.6,.3,.3,.1,.3,.1,.1]).all()
    assert (df[(dflt.stats_col, 'validation_accuracy', 'mean')] == [0.97, 0.89, 0.81, 0.8 , 0.65, 0.55, 0.46, 0.44, 0.19]).all()

    em.remove_previous_experiments (parent=True)

In [7]:
tst.run (test_do_query_and_show, tag='dummy')

running test_do_query_and_show
total data examined: 9 experiments with at least 5 runs done for each one
experiments: [5, 8, 2, 4, 7, 3, 1, 6, 0]
min experiment #: 0, max experiment #: 8
result of query:


parameters                      stats                                        \
      epochs offset validation_accuracy                                         
                                    max  mean    median       min num_results   
5       15.0    0.6            1.000000  0.97  1.000000  0.862921           5   
8       30.0    0.6            1.000000  0.89  0.839438  0.829499           5   
2        5.0    0.6            0.864282  0.81  0.802194  0.759965           5   
4       15.0    0.3            0.867930  0.80  0.812412  0.698613           5   
7       30.0    0.3            0.750076  0.65  0.656977  0.567965           5   
3       15.0    0.1            0.631286  0.55  0.523534  0.483821           5   
1        5.0    0.3            0.486164  0.46  0.459176  0.397024           5   
6       30.0    0.1            0.520651  0.44  0.419808  0.363639           5   
0        5.0    0.1            0.284362  0.19  0.189936  0.117181           5   

             
             
        std  
5  0.061304  
8  0.078095  
2  0.042088  
4  0.065922  
7  0.066409  
3  0.063784  
1  0.035920  
6  0.067830  
0  0.061893

### Change the metric that we want to show

In [8]:
#export tests.tools.test_query
def test_do_query_and_show_change_metric ():
    em = generate_data ('do_query_and_show_change_metric')
    
    do_query_and_show (metric='test_accuracy', manager_path=em.manager_path)
    
    em.remove_previous_experiments (parent=True)

In [9]:
tst.run (test_do_query_and_show_change_metric, tag='dummy')

running test_do_query_and_show_change_metric
total data examined: 9 experiments with at least 5 runs done for each one
experiments: [5, 8, 2, 4, 3, 7, 1, 0, 6]
min experiment #: 0, max experiment #: 8
result of query:


parameters                stats                                        \
      epochs offset test_accuracy                                         
                              max  mean    median       min num_results   
5       15.0    0.6      1.000000  0.93  0.914849  0.866905           5   
8       30.0    0.6      0.982601  0.86  0.859253  0.781303           5   
2        5.0    0.6      0.925193  0.83  0.842555  0.726914           5   
4       15.0    0.3      0.712835  0.61  0.592310  0.544079           5   
3       15.0    0.1      0.835273  0.61  0.535243  0.463074           5   
7       30.0    0.3      0.636414  0.58  0.570117  0.542417           5   
1        5.0    0.3      0.647555  0.52  0.526541  0.351243           5   
0        5.0    0.1      0.404256  0.30  0.320831  0.173696           5   
6       30.0    0.1      0.362067  0.29  0.285564  0.218306           5   

             
             
        std  
5  0.058813  
8  0.080142  
2  0.076029  
4  0.069167  
3  0.157971  
7  0.038304  
1  0.114638  
0  0.100413  
6  0.051178

### Run query with conditions

In [10]:
#export tests.tools.test_query
def test_do_query_and_show_with_conditions ():
    em = generate_data ('do_query_and_show_with_conditions')
    
    do_query_and_show (metric='validation_accuracy', op='max', pf={'epochs':15}, 
                       manager_path=em.manager_path)
    
    em.remove_previous_experiments (parent=True)

In [11]:
tst.run (test_do_query_and_show_with_conditions, tag='dummy')

running test_do_query_and_show_with_conditions
total data examined: 3 experiments with at least 5 runs done for each one
experiments: [5, 4, 3]
min experiment #: 3, max experiment #: 5
result of query:


parameters               stats                                        \
      offset validation_accuracy                                         
                             max  mean    median       min num_results   
5        0.6            1.000000  0.97  1.000000  0.862921           5   
4        0.3            0.867930  0.80  0.812412  0.698613           5   
3        0.1            0.631286  0.55  0.523534  0.483821           5   

             
             
        std  
5  0.061304  
4  0.065922  
3  0.063784

### Run query that sorts by maximum

In [12]:
#export tests.tools.test_query
def test_do_query_and_show_sort_maximum ():
    em = generate_data ('do_query_and_show_sort_maximum')
    
    do_query_and_show (metric='validation_accuracy', op='max', sort='max', 
                       stats=['mean', 'min', 'max'], manager_path=em.manager_path);
    
    em.remove_previous_experiments (parent=True)

In [13]:
tst.run (test_do_query_and_show_sort_maximum, tag='dummy')

running test_do_query_and_show_sort_maximum
total data examined: 9 experiments with at least 5 runs done for each one
experiments: [5, 8, 4, 2, 7, 3, 6, 1, 0]
min experiment #: 0, max experiment #: 8
result of query:


parameters                      stats                                    \
      epochs offset validation_accuracy                                     
                                    max  mean    median   min num_results   
5       15.0    0.6                1.00  0.97  1.000000  0.86           5   
8       30.0    0.6                1.00  0.89  0.839438  0.83           5   
4       15.0    0.3                0.87  0.80  0.812412  0.70           5   
2        5.0    0.6                0.86  0.81  0.802194  0.76           5   
7       30.0    0.3                0.75  0.65  0.656977  0.57           5   
3       15.0    0.1                0.63  0.55  0.523534  0.48           5   
6       30.0    0.1                0.52  0.44  0.419808  0.36           5   
1        5.0    0.3                0.49  0.46  0.459176  0.40           5   
0        5.0    0.1                0.28  0.19  0.189936  0.12           5   

             
             
        std  
5  0.061304  
8  0.078095  
4  0.065922  
2  0.042088  
7  0.066409  
3  0.063784  
6  0.067830  
1  0.035920  
0  0.061893

## parse_args

In [14]:
#export
def parse_args(args):
    default_always = ''

    parser = argparse.ArgumentParser(description='show metrics in visdom browser')
    # Datasets
    parser.add_argument('-m','--metric', type=str, default=None, help="metrics scores")
    parser.add_argument('--stats', type=str, nargs='+', default=['mean'],  help="statistics for multiple runs")
    parser.add_argument('--experiments', type=int, nargs='+', default=None,  help="experiment numbers")
    parser.add_argument('--folder', type=str, default=None)
    parser.add_argument('-v', type=str, default='{}', help='variable parameters')
    parser.add_argument('-f', type=str, default='{}', help='fixed parameters')
    parser.add_argument('-a', type=str, default='[]', help='all parameters')
    parser.add_argument('-e', '--exact', action= "store_true", help='exact match') 
    parser.add_argument('--last', type=int, default=None, help='include these last experiments') 
    parser.add_argument('--best', type=int, default=None, help='include these best experiments')
    parser.add_argument('--range', type=int, nargs='+', default=None, help='include this range of experiments')
    parser.add_argument('-c', '--compact', type=int, default=0, help='compact parameters to this number of characters') 
    parser.add_argument('--results', type=int, default=0, help='min number of results to consider') 
    parser.add_argument('-s', '--show', action= "store_true")
    parser.add_argument('--other', action= "store_true")
    parser.add_argument('--always', type=str, default = default_always)
    parser.add_argument('--op', default=None, type=str)
    parser.add_argument('--round', default=2, type=int, help='round scores to this number of digits')
    parser.add_argument('--runs', default=None, type=int, nargs='+', help='query restricted to run number provided')
    parser.add_argument('--sort', default=None, type=str)
    parser.add_argument('--width', default=None, type=int, help='max column width')
    parser.add_argument('-p', '--path', default=dflt.manager_path, type=str)
    pars = parser.parse_args(args)

    pars.v = eval(pars.v)
    pars.f = eval(pars.f)
    pars.a = eval(pars.a)
    pars.always = eval('dict(%s)' %pars.always)
    pars.f.update(pars.always)

    print (f'dictionary of query terms={pars.f}')
    
    return pars

## parse_arguments_and_query

In [15]:
#export
def parse_arguments_and_query (args):
    
    pars = parse_args(args)

    do_query_and_show (pall=pars.a, best = pars.best, compact = pars.compact, exact=pars.exact, 
                       experiments=pars.experiments, pf = pars.f, last=pars.last, 
                       metric = pars.metric, op = pars.op, other_parameters=pars.other,
                       input_range=pars.range, results=pars.results, folder= pars.folder, 
                       round=pars.round, runs = pars.runs, show=pars.show, stats=pars.stats,
                       pv = pars.v, sort=pars.sort, col_width=pars.width, manager_path=pars.path)

def main():
    parse_arguments_and_query (sys.argv[1:])

### Change the metric that we want to show

In [16]:
#export tests.tools.test_query
def test_parse_arguments_and_query_change_metric ():
    em = generate_data ('parse_arguments_and_query_change_metric')
    
    command = f'--metric test_accuracy -p {em.manager_path}'
    parse_arguments_and_query (command.split())
    
    em.remove_previous_experiments (parent=True)

In [17]:
tst.run (test_parse_arguments_and_query_change_metric, tag='dummy')

running test_parse_arguments_and_query_change_metric
dictionary of query terms={}
total data examined: 9 experiments with at least 5 runs done for each one
experiments: [5, 8, 2, 4, 3, 7, 1, 0, 6]
min experiment #: 0, max experiment #: 8
result of query:


parameters                stats                                        \
      epochs offset test_accuracy                                         
                              max  mean    median       min num_results   
5       15.0    0.6      1.000000  0.93  0.914849  0.866905           5   
8       30.0    0.6      0.982601  0.86  0.859253  0.781303           5   
2        5.0    0.6      0.925193  0.83  0.842555  0.726914           5   
4       15.0    0.3      0.712835  0.61  0.592310  0.544079           5   
3       15.0    0.1      0.835273  0.61  0.535243  0.463074           5   
7       30.0    0.3      0.636414  0.58  0.570117  0.542417           5   
1        5.0    0.3      0.647555  0.52  0.526541  0.351243           5   
0        5.0    0.1      0.404256  0.30  0.320831  0.173696           5   
6       30.0    0.1      0.362067  0.29  0.285564  0.218306           5   

             
             
        std  
5  0.058813  
8  0.080142  
2  0.076029  
4  0.069167  
3  0.157971  
7  0.038304  
1  0.114638  
0  0.100413  
6  0.051178

### Run query with conditions

In [18]:
#export tests.tools.test_query
def test_parse_arguments_and_query_with_conditions ():
    em = generate_data ('parse_arguments_and_query_with_conditions')

    command = f'--metric validation_accuracy --op max -f dict(epochs=15) -p {em.manager_path}'
    parse_arguments_and_query (command.split())

    em.remove_previous_experiments (parent=True)

In [19]:
tst.run (test_parse_arguments_and_query_with_conditions, tag='dummy')

running test_parse_arguments_and_query_with_conditions
dictionary of query terms={'epochs': 15}
total data examined: 3 experiments with at least 5 runs done for each one
experiments: [5, 4, 3]
min experiment #: 3, max experiment #: 5
result of query:


parameters               stats                                        \
      offset validation_accuracy                                         
                             max  mean    median       min num_results   
5        0.6            1.000000  0.97  1.000000  0.862921           5   
4        0.3            0.867930  0.80  0.812412  0.698613           5   
3        0.1            0.631286  0.55  0.523534  0.483821           5   

             
             
        std  
5  0.061304  
4  0.065922  
3  0.063784